In [136]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [137]:
df = pd.read_csv( "/home/bautista/Datos/Machine-Learning-Datos/Training.csv" )
df

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,...,Q2,2016,NaT,10,EUR,5272800.00,EUR,5272800.0,Closed Lost,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,...,Q1,2016,NaT,0,EUR,48230.00,EUR,48230.0,Closed Won,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,...,Q1,2016,NaT,0,USD,83865.60,USD,83865.6,Closed Won,Prod_Category_A_None
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,14,USD,7421881.50,USD,7421881.5,Closed Lost,Prod_Category_A_None
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,Q1,2018,NaT,25,USD,13357192.50,USD,13357192.5,Closed Lost,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16942,8781,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,...,Q1,2016,NaT,0,EUR,103350.00,EUR,299715.0,Closed Won,Prod_Category_A_None
16943,8786,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,...,Q2,2016,NaT,0,EUR,93015.00,EUR,299715.0,Closed Won,Prod_Category_A_None
16944,8792,EMEA,Austria,1,1,1,1,0,Bureaucratic_Code_4,1/15/2016,...,Q1,2016,NaT,0,EUR,103350.00,EUR,299715.0,Closed Won,Prod_Category_A_None
16945,28561,Americas,NE America,1,1,1,1,0,Bureaucratic_Code_4,10/20/2015,...,Q2,2016,NaT,4,USD,2346796.88,USD,0.0,Closed Lost,Prod_Category_A_None


# Conversion a Dolar

In [138]:
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = df['Total_Amount']*0.0096
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = df['Total_Amount']*1.17
df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = df['Total_Amount']*0.70
df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = df['Total_Amount']*1.29
df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USData'

In [139]:
short_df = df[['Region', 'Delivery_Year', 'Delivery_Quarter','TRF','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr','Stage' ]].rename(columns={'Stage': 'Decision'})
short_df = short_df[ (short_df['Decision'] == 'Closed Won') | (short_df['Decision'] == 'Closed Lost') ]

# Probanding

In [140]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [141]:
#X = short_df.drop( 'Decision', axis=1 )
#y = short_df['Decision']
#X = X.drop(columns='Total_Amount')
#X

In [142]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [143]:
X_training = short_df.drop(short_df[(short_df['Delivery_Year']== 2018) & (short_df['Delivery_Quarter'] == 'Q4')].index).drop(short_df[(short_df['Delivery_Year'] > 2018)].index)
Y_training = X_training.reset_index()['Decision']
X_training = X_training.reset_index().drop( ['Decision','index'], axis=1 )
X_training.count()

Region                                15083
Delivery_Year                         15083
Delivery_Quarter                      15083
TRF                                   15083
Pricing, Delivery_Terms_Approved      15083
Pricing, Delivery_Terms_Quote_Appr    15083
dtype: int64

In [144]:
Y_training.count()

15083

In [145]:
X_test = short_df[(short_df['Delivery_Year']== 2018) & (short_df['Delivery_Quarter'] == 'Q4')]
Y_test = X_test['Decision']
X_test = X_test.reset_index().drop( ['Decision','index'], axis=1 )
X_test.head()

,Region,Delivery_Year,Delivery_Quarter,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
0,EMEA,2018,Q4,6,1,1
1,APAC,2018,Q4,27,0,1
2,APAC,2018,Q4,27,0,1
3,APAC,2018,Q4,15,0,1
4,APAC,2018,Q4,4,0,1


In [146]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [147]:
print ('Training', X_training.shape, Y_training.shape)
print ('Test', X_test.shape, Y_test.shape)

Training (15083, 6) (15083,)
Test (1435, 6) (1435,)


# Encoder

In [148]:
ohe = OneHotEncoder(handle_unknown= 'ignore')
ohe.fit(X_training)
X_training_enc = ohe.transform(X_training)
X_test_enc = ohe.transform(X_test)

In [164]:
rfc = RandomForestClassifier(n_estimators=1000, max_depth = 5)
#n_estimators se refiere a la cantidad de arboles de decisiones que usa el rfc

In [165]:
rfc.fit(X_training_enc, Y_training)

RandomForestClassifier(max_depth=1, n_estimators=1000)

In [166]:
rfc_pred = rfc.predict(X_test_enc)

In [167]:
print(classification_report(Y_test, rfc_pred))
print(confusion_matrix(Y_test, rfc_pred))

              precision    recall  f1-score   support

 Closed Lost       0.71      0.07      0.13       640
  Closed Won       0.57      0.98      0.72       795

    accuracy                           0.57      1435
   macro avg       0.64      0.52      0.42      1435
weighted avg       0.63      0.57      0.45      1435

[[ 45 595]
 [ 18 777]]
